#### 📚 This is a [Geart Fork](https://github.com/geart891/RcloneLab/tree/geart891-patch-1), [Lastest version](https://colab.research.google.com/github/geart891/RLabClone/blob/master/blackpeal_template.ipynb)! [IMDB](https://www.imdb.com/),    [RARBG](https://rarbgway.org/torrents.php?order=seeders&by=DESC),    [MyAnimeList](https://myanimelist.net),    [Nyaa](https://nyaa.si/),    [BlackPearl](https://blackpearl.biz/), [LinkSafe](https://links.blackpearl.biz)


In [0]:
from google.colab import drive
drive.mount('/content/drive')

# <img src='https://i.imgur.com/N4bdPff.png' height="30" alt="IMDB"/> **Generate IMDB template**

In [0]:
# ============================= FORM ============================= #
# @markdown ###IMDB Informations
IMDB_URL_OR_ID = ""  # @param {type:"string"}
PATH = ""  # @param {type:"string"}
LINK = ""  # @param {type:"string"}
SCREENSHOTS = ""  # @param {type:"string"}
TRAILER = ""  # @param {type:"string"}
# @markdown ---
# @markdown ###Forum Requirements
LIKE_REQUIREMENT = 35  # @param {type: "number"}
POST_REQUIREMENT = 35  # @param {type: "number"}
DAY_REQUIREMENT = 25  # @param {type: "number"}
THANK_REQUIREMENT = True  # @param {type:"boolean"}
# ============================= FORM ============================= #

import json, re, requests, sys, os, time  # pylint: disable=import-error
from IPython.display import clear_output  # pylint: disable=import-error
from IPython import get_ipython  # pylint: disable=import-error
from bs4 import BeautifulSoup  # pylint: disable=import-error

if os.path.isfile("/usr/bin/mediainfo") != True:
    get_ipython().system_raw("apt install mediainfo")
if os.path.isdir("/content/drive") != True:
    from google.colab import drive  # pylint: disable=import-error

    drive.mount("/content/drive")

RQ_HEADERS = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.8; rv:21.0) Gecko/20100101 Firefox/21.0"
}
API_KEY = "991109a2"


def getImdbID():
    if IMDB_URL_OR_ID == "":
        print("Need IMDB ID")
        sys.exit()
    imdb_pattern = re.compile(
        "(tt\d{7,8})"  # pylint: disable=anomalous-backslash-in-string
    )
    return re.findall(imdb_pattern, IMDB_URL_OR_ID)[0]


def getStoryline(imdb_id):
    r = requests.get(
        f"https://www.imdb.com/title/{imdb_id}",
        headers=RQ_HEADERS,
        verify=False,  # nosec
    )
    clear_output()
    soup = BeautifulSoup(r.text, "html.parser")
    return soup.find_all("div", class_="inline canwrap")[0].find("span").text.lstrip()


def outputImdbTemplate(imdb_id):
    imdb_url = f"http://www.omdbapi.com/?i={imdb_id}&apikey={API_KEY}&r=json&plot=full"
    if TRAILER != "":
        youtube_pattern = re.compile(
            "(https?://)?(www\.)?(youtube|youtu|youtube-nocookie)\.(com|be)/(watch\?v=|embed/|v/|.+\?v=)?(?P<id>[A-Za-z0-9\-=_]{11})"  # pylint: disable=anomalous-backslash-in-string
        )
        youtube_match = re.match(youtube_pattern, TRAILER)
        if not youtube_match:
            clear_output()
            print("Your trailer link is invalid. Please only put in youtube link!")
            sys.exit()
        else:
            trailerEnabled = True
    else:
        trailerEnabled = False

    if SCREENSHOTS != "":
        screenshot_list = SCREENSHOTS.split(" ")
        screenshot_enabled = True
    else:
        screenshot_enabled = False

    if PATH == "":
        mediainfoEnabled = False
    else:
        mediainfoEnabled = True
        get_ipython().system_raw(  # pylint: disable=undefined-variable
            f'mediainfo --Logfile=/root/.nfo "{PATH}"'
        )
        time.sleep(5)  # Wait 5s for the nfo file to populate properly
        with open("/root/.nfo", "r", encoding="utf-8") as nfo_file:
            nfo = nfo_file.readlines()
            del nfo[1]
            size = next(filter(lambda x: x[:9] == "File size", nfo), None)[43:].strip()
            duration = next(filter(lambda x: x[:8] == "Duration", nfo), None)[
                43:
            ].strip()
            nfo[
                1
            ] = f"{'Title name'.ljust(41)[:41]}: {os.path.splitext(os.path.basename(PATH))[0].replace('.', ' ')} [{size}] [{duration}]\n"
            nfo = "".join(nfo)

    if LINK == "":
        print("You forgot putting in the link to download!")
        sys.exit()

    # Fetch data
    imdb_data = json.loads(requests.get(imdb_url, verify=False).text)  # nosec
    # Process data
    text_dump = f"[center][img width=\"350px\"]{imdb_data['Poster']}[/img]\n\
                  [url=https://www.imdb.com/title/{imdb_id}][font=tahoma][color=rgb(250, 197, 28)][b][size=7]{imdb_data['Title']} ({imdb_data['Year']})[/size][/b][/color][/font][/url]\n\
                  [url=https://www.imdb.com/title/{imdb_id}][img height=\"16px\"]https://i.imgur.com/rcSipDw.png[/img][/url][size=6][b] {imdb_data['imdbRating']}[/b]/10 --- [img height=\"16px\"]https://i.imgur.com/sEpKj3O.png[/img]{imdb_data['imdbVotes']}[/size]\
                [/center]\n\
                [hr][/hr]\
                  [indent][size=6][font=tahoma][color=rgb(84, 172, 210)][b]Plot Summary[/b][/color][/font][/size][/indent]\n \n \
                {getStoryline(imdb_id)}\n"

    if trailerEnabled:
        text_dump += f"[hr][/hr]\
                     [indent][size=6][font=tahoma][color=rgb(84, 172, 210)][b]Trailer[/b][/color][/font][/size][/indent]\n\
                   [center][media=youtube]{youtube_match.group(6)}[/media][/center]\n"

    if screenshot_enabled:
        text_dump += f"[hr][/hr]\
                     [indent][size=6][font=tahoma][color=rgb(84, 172, 210)][b]Screenshots[/b][/color][/font][/size][/indent]\n\
                   [spoiler='Click here to view screenshots']\n"

        for i in screenshot_list:
            text_dump += f"[img]{i}[/img]\n"
        text_dump += "[/spoiler]"
    text_dump += f"[hr][/hr]\
                   [indent][size=6][font=tahoma][color=rgb(84, 172, 210)][b]Movie Info[/b][/color][/font][/size][/indent]\n\
                 [LIST]\
                   [*][b][color=rgb(251, 160, 38)]Rating :[/b][/color] {imdb_data['Rated']}\n\
                   [*][b][color=rgb(251, 160, 38)]Genre :[/b][/color] {imdb_data['Genre']}\n\
                   [*][b][color=rgb(251, 160, 38)]Directed By :[/b][/color] {imdb_data['Director']}\n\
                   [*][b][color=rgb(251, 160, 38)]Written By :[/b][/color] {imdb_data['Writer']}\n\
                   [*][b][color=rgb(251, 160, 38)]Starring :[/b][/color] {imdb_data['Actors']}\n\
                   [*][b][color=rgb(251, 160, 38)]Release Date :[/b][/color] {imdb_data['Released']}\n\
                   [*][b][color=rgb(251, 160, 38)]Runtime :[/b][/color] {imdb_data['Runtime'] if 'Runtime' in imdb_data.keys() else 'N/A'}\n\
                   [*][b][color=rgb(251, 160, 38)]On Disk/Streaming :[/b][/color] {imdb_data['DVD']}\n\
                   [*][b][color=rgb(251, 160, 38)]Box Office :[/b][/color] {imdb_data['BoxOffice']}\n\
                   [*][b][color=rgb(251, 160, 38)]Awards :[/b][/color] {imdb_data['Awards']}\n\
                   [*][b][color=rgb(251, 160, 38)]Studio :[/b][/color] {imdb_data['Production'] if 'Production' in imdb_data.keys() else 'N/A'}\
                 [/LIST]\n"

    if mediainfoEnabled:
        text_dump += f"[hr][/hr]\
                     [indent][size=6][font=tahoma][color=rgb(84, 172, 210)][b]Media Info[/b][/color][/font][/size][/indent]\n\
                   [spoiler='Click here to view Media Info'][code]{nfo}[/code][/spoiler]\n"

    text_dump += f"""\
        [hr][/hr]
        [center][size=6][font=tahoma][color=rgb(97, 189, 109)][b][u]Download Link[/u][/b][/color][/font][/size]
            {'[hidereact=1,2,4]' if THANK_REQUIREMENT else ''}
                {f'[hidereactscore={LIKE_REQUIREMENT}]' if LIKE_REQUIREMENT != 0 else ''}
                    {f'[hideposts={POST_REQUIREMENT}]' if POST_REQUIREMENT != 0 else ''}
                        {f'[days={DAY_REQUIREMENT}]' if DAY_REQUIREMENT != 0 else ''}
                            [downcloud]{LINK}[/downcloud]                        
                        {'[/days]' if DAY_REQUIREMENT != 0 else ''}
                    {'[/hideposts]' if POST_REQUIREMENT != 0 else ''}
                {'[/hidereactscore]' if LIKE_REQUIREMENT != 0 else ''}
            {'[/hidereact]' if THANK_REQUIREMENT else ''}
        [/center]\
        """

    return text_dump


print(outputImdbTemplate(getImdbID()))


# <img src='https://i.imgur.com/e5Thsob.jpg?1' height="25" alt="MAL"/> **Generate MAL template**

In [0]:
# ============================= FORM ============================= #
# @markdown ###Anime Information
ANIME_URL_OR_ID = ""  # @param {type:"string"}
PATH = ""  # @param {type:"string"}
LINK = ""  # @param {type:"string"}
SCREENSHOTS = ""  # @param {type:"string"}
TRAILER = ""  # @param {type:"string"}
# @markdown ---
# @markdown ###Forum Requirements
LIKE_REQUIREMENT = 35  # @param {type: "number"}
POST_REQUIREMENT = 35  # @param {type: "number"}
DAY_REQUIREMENT = 25  # @param {type: "number"}
THANK_REQUIREMENT = True  # @param {type:"boolean"}
# ============================= FORM ============================= #

import re, time, sys, os
from bs4 import BeautifulSoup  # pylint: disable=import-error
from IPython import get_ipython  # pylint: disable=import-error

if os.path.isfile("/usr/bin/mediainfo") != True:
    get_ipython().system_raw("apt install mediainfo")
    get_ipython().system_raw("pip install git+git://github.com/AWConant/jikanpy.git")
if os.path.isdir("/content/drive") != True:
    from google.colab import drive  # pylint: disable=import-error

    drive.mount("/content/drive")

from jikanpy import Jikan  # pylint: disable=import-error

jikan = Jikan()


def getMalID():
    if ANIME_URL_OR_ID == "":
        print("Need MAL ID")
        sys.exit()
    mal_pattern = re.compile(
        "(anime)?/(\d{1,7})/(.+)?"  # pylint: disable=anomalous-backslash-in-string
    )
    if re.match(r"\d+", ANIME_URL_OR_ID):
        mal_id = ANIME_URL_OR_ID
    else:
        mal_id = re.findall(mal_pattern, ANIME_URL_OR_ID)[0][1]
    return mal_id


def get_info(anime_data, info):
    text = ""
    for i in range(len(anime_data[f"{info}"])):
        text += anime_data[info][i]["name"] + ", "
    return text[:-2]


def get_mal_template(mal_id):
    # Fetch data
    anime_data = jikan.anime(mal_id)
    youtube_pattern = re.compile(  # pylint: disable=anomalous-backslash-in-string
        "(https?://)?(www\.)?(youtube|youtu|youtube-nocookie)\.(com|be)/(watch\?v=|embed/|v/|.+\?v=)?(?P<id>[A-Za-z0-9\-=_]{11})"  # pylint: disable=anomalous-backslash-in-string
    )
    if SCREENSHOTS != "":
        screenshot_list = SCREENSHOTS.split(" ")
        screenshot_enabled = True
    else:
        screenshot_enabled = False

    if PATH == "":
        mediainfo_enabled = False
    else:
        mediainfo_enabled = True
        get_ipython().system_raw(f'mediainfo --Logfile=/root/.nfo "{PATH}"')
        time.sleep(5)  # Wait 5s for the nfo file to populate properly
        with open("/root/.nfo", "r", encoding="utf-8") as nfo_file:
            nfo = nfo_file.readlines()
        del nfo[1]
        size = next(
            filter(
                lambda x: x[:43] == "File size                                : ", nfo
            ),
            None,
        )
        duration = next(
            filter(
                lambda x: x[:43] == "Duration                                 : ", nfo
            ),
            None,
        )
        nfo[
            1
        ] = f"{'Title name'.ljust(41)[:41]}: {os.path.splitext(os.path.basename(PATH))[0].replace('.', ' ')} [{size[43:].strip()}] [{duration[43:].strip()}]\n"
        nfo = "".join(nfo)

    if LINK == "":
        print("You forgot putting in the link to download!")
        sys.exit()

    # Process data
    text_dump = f"[center]\
        [img]{anime_data['image_url']}[/img]\n\
        [url={anime_data['url']}][font=tahoma][color=rgb(250, 197, 28)][b][size=7]{anime_data['title']} ({anime_data['type']} - {anime_data['premiered']})[/size][/b][/color][/font][/url]\n\
        [url={anime_data['url']}][img height=\"16px\"]https://i.imgur.com/YJgzM4V.png[/img][/url][size=6][b] {anime_data['score']}[/b]/10 --- [img height=\"16px\"]https://i.imgur.com/sEpKj3O.png[/img]{anime_data['scored_by']}[/size]\
        [/center]\n\
        [hr][/hr]\
        [indent][font=tahoma][size=6][color=rgb(84, 172, 210)][b]Synopsis[/b][/color][/size][/font][/indent]\n \n{anime_data['synopsis']}\n"

    try:
        text_dump += f"[hr][/hr]\
            [indent][font=tahoma][size=6][color=rgb(84, 172, 210)][b]Trailer[/b][/color][/size][/font][/indent]\n\
            [center][media=youtube]{re.match(youtube_pattern, str(anime_data['trailer_url'])).group(6)}[/media][/center]\n"

    except:  # nosec
        pass

    if screenshot_enabled:
        text_dump += f"[hr][/hr]\
            [indent][size=6][font=tahoma][color=rgb(84, 172, 210)][b]Screenshots[/b][/color][/font][/size][/indent]\n\
            [spoiler='Click here to view screenshots']\n"

        for i in screenshot_list:
            text_dump += f"[img]{i}[/img]\n"
        text_dump += "[/spoiler]"

    text_dump += f"[hr][/hr]\
        [indent][size=6][font=tahoma][color=rgb(84, 172, 210)][b]Anime Info[/b][/color][/font][/size][/indent]\n\
        [LIST]\
        [*][b][color=rgb(251, 160, 38)]Rating :[/b][/color] {anime_data['rating']}\n\
        [*][b][color=rgb(251, 160, 38)]Type :[/b][/color] {anime_data['type']}\n\
        [*][b][color=rgb(251, 160, 38)]Episodes :[/b][/color] {anime_data['episodes']}\n\
        [*][b][color=rgb(251, 160, 38)]Status :[/b][/color] {anime_data['status']}\n\
        [*][b][color=rgb(251, 160, 38)]Aired :[/b][/color] {anime_data['aired']['string']}\n\
        [*][b][color=rgb(251, 160, 38)]Premiered :[/b][/color] {anime_data['premiered']}\n\
        [*][b][color=rgb(251, 160, 38)]Producers :[/b][/color] {get_info(anime_data, 'producers')}\n\
        [*][b][color=rgb(251, 160, 38)]Studios :[/b][/color] {get_info(anime_data, 'studios')}\n\
        [*][b][color=rgb(251, 160, 38)]Source :[/b][/color] {anime_data['source']}\n\
        [*][b][color=rgb(251, 160, 38)]Genres :[/b][/color] {get_info(anime_data, 'genres')}\n\
        [*][b][color=rgb(251, 160, 38)]Duration :[/b][/color] {anime_data['duration']}\
        [/LIST]\n"

    if mediainfo_enabled:
        text_dump += f"[hr][/hr]\
            [indent][size=6][font=tahoma][color=rgb(84, 172, 210)][b]Media Info[/b][/color][/font][/size][/indent]\n\
            [spoiler='Click here to view Media Info'][code]{nfo}[/code][/spoiler]\n"

    text_dump += f"""\
        [hr][/hr]
        [center][size=6][font=tahoma][color=rgb(97, 189, 109)][b][u]Download Link[/u][/b][/color][/font][/size]
            {'[hidereact=1,2,4]' if THANK_REQUIREMENT else ''}
                {f'[hidereactscore={LIKE_REQUIREMENT}]' if LIKE_REQUIREMENT != 0 else ''}
                    {f'[hideposts={POST_REQUIREMENT}]' if POST_REQUIREMENT != 0 else ''}
                        {f'[days={DAY_REQUIREMENT}]' if DAY_REQUIREMENT != 0 else ''}
                            [downcloud]{LINK}[/downcloud]                        
                        {'[/days]' if DAY_REQUIREMENT != 0 else ''}
                    {'[/hideposts]' if POST_REQUIREMENT != 0 else ''}
                {'[/hidereactscore]' if LIKE_REQUIREMENT != 0 else ''}
            {'[/hidereact]' if THANK_REQUIREMENT else ''}
        [/center]\
        """

    return text_dump


print(get_mal_template(getMalID()))
